In [1]:
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
from datetime import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


In [2]:
# Parameters
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
COLUMNS = ['Close', 'SMA200', 'SMA100', 'SMA50', 'SMA13', 'RSI14']
LABEL = 'signal'
seq_length = 30
batch_size = 4096
epochs = 100
dropout = 0.4
learning_rate = 0.01
random_state = 13
train_test_ratio = 0.2

input_size = len(COLUMNS)  # ['Close', 'SMA200', 'SMA50', 'RSI14']

num_classes = 3  # [buy, sell, nothing]

X, y = [], []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data = pd.read_csv(FILE_PATH)

In [4]:
data = data.dropna() 
# data = data.round(4)
data[LABEL] = data[LABEL] - 1 

In [5]:
features = data[COLUMNS].values
labels = data[LABEL].values

In [6]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [7]:
for i in range(seq_length, len(features_scaled)):
    X.append(features_scaled[i-seq_length:i])
    y.append(labels[i])
X = np.array(X)
y = np.array(y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=train_test_ratio, random_state=random_state)

In [9]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

In [10]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

In [12]:
class CNNLSTM(nn.Module):
    def __init__(self, num_features, num_classes):
        super(CNNLSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch, features, sequence)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.permute(0, 2, 1)  # Change shape to (batch, sequence, features)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Take the last output of LSTM
        x = self.fc(x)
        return x

In [13]:
model = CNNLSTM(num_features=input_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
for epoch in tqdm(range(epochs), desc="Training Epochs"):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    
    val_accuracy = correct / total
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, '
          f'Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}')

    # Save Model
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    torch.save(model.state_dict(), f"D:/Programing/AI Trader/Model/cnnLstmModelv.1.1/Model v.1.1_Train-loss {train_loss/len(train_loader):.4f}_Val-loss {val_loss/len(val_loader):.4f}_Val-Accuracy {val_accuracy:.4f}_at {timestamp}.model")


Training Epochs:   1%|          | 1/100 [00:04<07:43,  4.68s/it]

Epoch 1/100, Train Loss: 0.2058, Val Loss: 1.4148, Val Accuracy: 0.6812


Training Epochs:   2%|▏         | 2/100 [00:08<06:26,  3.94s/it]

Epoch 2/100, Train Loss: 0.2176, Val Loss: 1.3816, Val Accuracy: 0.6826


Training Epochs:   3%|▎         | 3/100 [00:11<06:04,  3.75s/it]

...

Epoch 98/100, Train Loss: 0.1941, Val Loss: 1.4245, Val Accuracy: 0.6817


Training Epochs:  99%|█████████▉| 99/100 [05:57<00:03,  3.61s/it]

Epoch 99/100, Train Loss: 0.1752, Val Loss: 1.4658, Val Accuracy: 0.6751


Training Epochs: 100%|██████████| 100/100 [06:01<00:00,  3.61s/it]

Epoch 100/100, Train Loss: 0.1975, Val Loss: 1.7128, Val Accuracy: 0.6514


In [15]:
# torch.cuda.empty_cache()

# device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')


# import numpy as np
# import pandas as pd
# import torch
# from sklearn.metrics import classification_report

# # Load the backtesting data


# FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
# data = pd.read_csv(FILE_PATH)
# backtest_data = FILE_PATH
# features = data[COLUMNS].values
# labels = data[LABEL].values

# # Normalize features using the same scaler as in training
# features_scaled = scaler.fit_transform(features)

# # Prepare sequences for backtesting
# seq_length = 30
# X_test, y_test = [], []
# for i in range(seq_length, len(features_scaled)):
#     X_test.append(features_scaled[i-seq_length:i])
#     y_test.append(labels[i])

# X_test = np.array(X_test)
# y_test = np.array(y_test)

# # Convert to PyTorch tensors
# X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
# y_test = torch.tensor(y_test, dtype=torch.long).to(device)

# # Load the trained model
# model = CNNLSTM(num_features=4, num_classes=3)
# MODEL_PATH = "D:/Programing/AI Trader/Model/cnnLstmModelv.1.0/Model v.1.0_Train-loss 0.2628_Val-loss 0.9739_Val-Accuracy 0.7121_at 20241227-175258.model"
# model.load_state_dict(torch.load(MODEL_PATH))
# model.to(device)
# model.eval()
# # Generate predictions
# with torch.no_grad():
#     outputs = model(X_test)
#     predictions = torch.argmax(outputs, dim=1).cpu().numpy()

# # Evaluate model performance
# print("Classification Report:")
# print(classification_report(y_test.cpu().numpy(), predictions))

# # Backtest simulation
# initial_balance = 10000  # Starting balance
# balance = initial_balance
# position = None  # 'long' or 'short'
# profit_loss = []

# for i, prediction in enumerate(predictions):
#     if prediction == 0:  # Buy signal
#         if position != 'long':
#             balance += (features[i, 0] - features[i-1, 0]) if position == 'short' else 0
#             position = 'long'
#     elif prediction == 1:  # Sell signal
#         if position != 'short':
#             balance += (features[i-1, 0] - features[i, 0]) if position == 'long' else 0
#             position = 'short'
#     elif prediction == 2:  # Do nothing
#         pass
#     profit_loss.append(balance)

# # Calculate backtest metrics
# total_profit = balance - initial_balance
# win_rate = sum([1 for pl in profit_loss if pl > 0]) / len(profit_loss)
# max_drawdown = min(profit_loss) / initial_balance
# profit_factor = sum([pl for pl in profit_loss if pl > 0]) / abs(sum([pl for pl in profit_loss if pl < 0]))

# print(f"Backtest Results:")
# print(f"Total Profit: ${total_profit:.2f}")
# print(f"Win Rate: {win_rate * 100:.2f}%")
# print(f"Max Drawdown: {max_drawdown:.2f}")
# print(f"Profit Factor: {profit_factor:.2f}")

# # Plot equity curve
# import matplotlib.pyplot as plt

# plt.plot(profit_loss)
# plt.title('Equity Curve')
# plt.xlabel('Trade')
# plt.ylabel('Balance')
# plt.show()


In [16]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

# # Calculate confusion matrix
# conf_matrix = confusion_matrix(y_test.cpu().numpy(), predictions)

# # Plot confusion matrix
# disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['Buy', 'Sell', 'Nothing'])
# disp.plot(cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.show()

# # Calculate F1 score
# f1 = f1_score(y_test.cpu().numpy(), predictions, average='weighted')
# print(f"F1 Score (Weighted): {f1:.4f}")
